# Lesson 3 - Introduction

This lesson is primarily about tools. We will be covering two tools: *language* and *functions*.

## Language

Python has a syntax for statements, expressions, classes with operator overloading (`x + y`), plus some smaller, specialized languages, like string formatting. In general, one can develop a domain-specific language to represent and solve specific problems. This is what we will do in this unit. To this end, we will describe what a *language* is, what a *grammar* is, the difference between a *compiler* and an *interpreter*, and how to use languages as a design tool.

## Regular Expressions

REs are an example of a language that can be expressed as strings. For example `'a*b*c*'`. To make sense of inputs like these we need to make sense of what the possible *grammars* are and what are the possible *languages* that those grammars correspond to. Regular Expression operate on strings and use patterns represented by strings, but they can be fairly complicated, have nested structures that make them look more like trees than like strings. In the rest of this unit we will be dealing with an internal representation of REs based on trees that will be different from the one above. The input will still be in the form of strings, but they will be internally represented as trees, and we will be dealing with trees from now on.

A **grammar** is a description of a language and a **language** is a set of strings. In our example above `'a*b*c*` is the grammar and strings like `abc`, `aaabbcc`, `ccccc` are the language associated with the grammar.

 Representing grammars as strings, like `'a*b*c*` or `'a+b?` is convenient for small expressions, but it can become complex with longer ones. We are going to use a representation that is more **compositional**. We are going to describe an Application Programming Interface (API, as opposed to the UI), i.e., a series of function calls that can be used to describe the grammar of a RE.
 
 We consider the following building blocks:

 - The API call `lit(s)` is the **literal** of string `s`. `lit('a')` describes the language consisting only of character string `'a'`, i.e., {'a'}, and nothing else.
 - The API call `seq(x, y)` is the **sequence** of x and y. `seq(lit(a), lit(b))` would describe the language consisting only of the string `'ab'`, i.e. {ab}.
 - The API call `alt(x, y)` stands for **alternatives**. `seq(lit(a), lit(b))` would correspond of two possibilities: either `a` or `b`, i.e., {a, b}.
 - The API call `star(x)` means **zero or more repetitions** of `'x'`, and would therefore correspond to {a, aa, aaa,...} and so on.
 - The API call `oneof(s)` is the same as `alt(c1, c2, ...)` where `'c1'` etc. are the characters string `s` is made of. `oneof('abc')` matches {a, b, c}.
 - The API call `eol` means **end of line* matches only the end of a character string and nothing else, so it matches the empty string {''}, but only at the end. `seq(lit('a'), eol)` matches {a}.
 - The API call `dot` matches any possible character {a, b, c,...}
 - The API call `plus(s)` means **one or more repetitions** of `'x'`. **NOTE** it does not seem to be implemented in this unit.

Python has the `re` module, but the point of this lesson is not to learn how to use REs, but rather how to build a **language processor**. Let's start writing a test function we can test our code against. We haven't yet defined any of the functions referred to in the tests.

In [2]:
def test_search():
    a, b, c = list('a'), list('b'), list('c')
    abcstars = seq(star(a), seq(star(b), star(c)))
    dotstar = star(dot)
    assert search(lit('def'), 'abcdefg') == 'def'
    assert search(seq('def', eol), 'abcdef') == 'def'
    assert search(seq(lit('def'), eol), 'abcdefg') == None
    assert search(a, 'not the start') == 'a'
    assert match(a, 'not the start') == None
    assert match(abcstars, 'aaabbbccccccccdef') == 'aaabbbcccccccc'
    assert match(abcstars, 'junk') == ''
    assert all(match(seq(abcstars, eol), s) == s for s in 'abc aaabbccc aaaabcccc'.split())
    assert all(match(seq(abcstars, eol), s) == None for s in 'cab aaabbcccd aaaa-b-cccc'.split())
    r = seq(lit('ab'), seq(dotstar, seq(list('aca'), seq(dotstar, seq(a, eol)))))
    assert all(search(r, s) is not None for s in 'abracadabra abacaa about-acacia-flora'.split())
    assert all(match(seq(c, seq(dotstar, b)), s) is not None for s in 'cab cob carob cb carbuncle'.split())
    assert not any(match(seq(c, seq(dot, b)), s) for s in 'carb cb across scab'.split())
    return 'test_search passes'

We will cover a subset of regular expression in this lesson: `*`, `?`, `.`, `^`, `$`, `''` (the empty string, which only matches an empty string). The functions in `test_search()` are based on [Rob Pike's regular expression matcher](https://www.cs.princeton.edu/courses/archive/spr09/cos333/beautiful.html).

In the tests above we are using two functions: 

- `search(pattern, text)`: returns a string that is the earliest match of the pattern in the text, and if there is more than one match in the same location, it will be the longest of those.
- `match(pattern, text)`: matches only if the pattern occurs at the very start of `text`, so `match('def', 'abcdef')` would return `None`.

Below we provide the implementations of `search()` and `match()`, plus a couple of utility functions. Of particular interest is the structure of `match_star(p, pattern, text)`, which uses a clever recursive call.

In [5]:
def match1(p, text):
    """Return True if first character of text matches pattern character p."""
    if not text:
        return False
    return p == '.' or p == text[0]

def match_star(p, pattern, text):
    """Return True if any number of char p, followed by pattern,
    matches text."""
    return (match(pattern, text) or  # match zero times
            (match1(p, text) and     # match exactly one time
             match_star(p, pattern, text[1:])))

def search(pattern, text):
    "Return True if pattern appears anywhere in text."
    if pattern.startswith('^'):
        return match(pattern[1:], text)
    else:
        return match('.*' + pattern, text)

def match(pattern, text):
    "Return True if pattern appears at the start of text."
    if pattern == '':
        return True
    elif pattern == '$':
        return (text == '')
    elif len(pattern) > 1 and pattern[1] in '*?':
        p, op, pat = pattern[0], pattern[1], pattern[2:]
        if op == '*':
            return match_star(p, pat, text)
        elif op == '?':
            if match1(p, text) and match(pat, text[1:]):
                return True
            else:
                return match(pat, text)
        else:
            return (match1(pattern[0], text) and
                    match(pattern[1:], text[1:]))

## Concept Inventory

These are the concept we need to consider in this unit:

- Patterns.
- Texts we want to match the patterns against, and the result of this mathing.

We will also need these concepts

- A concept of *partial result*.
- A notion of *control over iteration*.

To understand why these additional notions are needed, let's consider the following example: our pattern is `'a*b+'` and our text is `'aaab'`. If our pattern is represented in our API as `seq(star(a), plus(lit(b)))`, the first part, `star(a)` would match the first character. If we then look for the second part of the pattern, this would not match. We would then need a mechanism to go back, and check whether `'aa'` is a pattern, and so on. Similarly, if we need to evaluate between alternatives, we need some form of control over this form of iteration.

### `matchset(pattern, text)`

It is difficult to see it now, but if we choose to represent these partial results as a *set of remainders of the text*, where by *remainder* we mean everything left after the match, then everything falls in place, and most of the perceived difficulty of dealing with such cases goes away. We are therefore going to introduce a function called `matchset(pattern, text)` that returns this set of remainders. For example `matchset(star(lit(a)), text='aaab')` the output would be the set `{aaab, aab, ab, b}`.

In [12]:
#----------------
# User Instructions
#
# The function, matchset, takes a pattern and a text as input
# and returns a set of remainders. For example, if matchset 
# were called with the pattern star(lit(a)) and the text 
# 'aaab', matchset would return a set with elements 
# {'aaab', 'aab', 'ab', 'b'}, since a* can consume one, two
# or all three of the a's in the text.
#
# Your job is to complete this function by filling in the 
# 'dot' and 'oneof' operators to return the correct set of 
# remainders.
#
# dot:   matches any character.
# oneof: matches any of the characters in the string it is 
#        called with. oneof('abc') will match a or b or c.

def matchset(pattern, text):
    "Match pattern at start of text; return a set of remainders of text."
    op, x, y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]])
    elif 'oneof' == op:
        return set([text[text.find(x)+1:]])
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                    for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: %s' % pattern)

null = frozenset()

def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y

def test():
    assert matchset(('lit', 'abc'), 'abcdef')            == set(['def'])
    assert matchset(('seq', ('lit', 'hi '),
                     ('lit', 'there ')), 
                   'hi there nice to meet you')          == set(['nice to meet you'])
    assert matchset(('alt', ('lit', 'dog'), 
                    ('lit', 'cat')), 'dog and cat')      == set([' and cat'])
    assert matchset(('dot',), 'am i missing something?') == set(['m i missing something?'])
    assert matchset(('oneof', 'a'), 'aabc123')           == set(['abc123'])
    assert matchset(('eol',),'')                         == set([''])
    assert matchset(('eol',),'not end of line')          == frozenset([])
    assert matchset(('star', ('lit', 'hey')), 'heyhey!') == set(['!', 'heyhey!', 'hey!'])
    
    return 'tests pass'

print(test())

tests pass


## Filling Out The API

In the code below we provide the implementations of the various API calls.

In [19]:
#---------------
# User Instructions
#
# Fill out the API by completing the entries for alt, 
# star, plus, and eol.


def lit(string):  return ('lit', string)
def seq(x, y):    return ('seq', x, y)
def alt(x, y):    return ('alt', x, y)
def star(x):      return ('star', x)
def plus(x):      return ('seq', x, ('star', x))
def opt(x):       return alt(lit(''), x) #opt(x) means that x is optional
def oneof(chars): return ('oneof', tuple(chars))
dot = ('dot',)
eol = ('eol',)

def test():
    assert lit('abc')         == ('lit', 'abc')
    assert seq(('lit', 'a'), 
               ('lit', 'b'))  == ('seq', ('lit', 'a'), ('lit', 'b'))
    assert alt(('lit', 'a'), 
               ('lit', 'b'))  == ('alt', ('lit', 'a'), ('lit', 'b'))
    assert star(('lit', 'a')) == ('star', ('lit', 'a'))
    assert plus(('lit', 'c')) == ('seq', ('lit', 'c'), 
                                  ('star', ('lit', 'c')))
    assert opt(('lit', 'x'))  == ('alt', ('lit', ''), ('lit', 'x'))
    assert oneof('abc')       == ('oneof', ('a', 'b', 'c'))
    return 'tests pass'

print(test())

tests pass


### Search and Match

In [55]:
#---------------
# User Instructions
#
# Complete the search and match functions. Match should
# match a pattern only at the start of the text. Search
# should match anywhere in the text.

def search(pattern, text):
    "Match pattern anywhere in text; return longest earliest match or None."
    for i in range(len(text)):
        m = match(pattern, text[i:])
        if m:
            return m
        
def match(pattern, text):
    "Match pattern against start of text; return longest match found or None."
    remainders = matchset(pattern, text)
    if remainders:
        shortest = min(remainders, key=len)
        return text[:text.find(shortest)] if len(text) > 1 else text
    
def components(pattern):
    "Return the op, x, and y arguments; x and y are None if missing."
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y

def matchset(pattern, text):
    "Match pattern at start of text; return a set of remainders of text."
    op, x, y = components(pattern)
    if 'lit' == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif 'seq' == op:
        return set(t2 for t1 in matchset(x, text) for t2 in matchset(y, t1))
    elif 'alt' == op:
        return matchset(x, text) | matchset(y, text)
    elif 'dot' == op:
        return set([text[1:]]) if text else null
    elif 'oneof' == op:
        return set([text[1:]]) if text.startswith(x) else null
    elif 'eol' == op:
        return set(['']) if text == '' else null
    elif 'star' == op:
        return (set([text]) |
                set(t2 for t1 in matchset(x, text)
                    for t2 in matchset(pattern, t1) if t1 != text))
    else:
        raise ValueError('unknown pattern: %s' % pattern)
    
null = frozenset()

def lit(string):  return ('lit', string)
def seq(x, y):    return ('seq', x, y)
def alt(x, y):    return ('alt', x, y)
def star(x):      return ('star', x)
def plus(x):      return seq(x, star(x))
def opt(x):       return alt(lit(''), x)
def oneof(chars): return ('oneof', tuple(chars))
dot = ('dot',)
eol = ('eol',)

def test():
    assert match(('star', ('lit', 'a')),'aaabcd') == 'aaa'
    assert match(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == None
    assert match(('alt', ('lit', 'b'), ('lit', 'a')), 'ab') == 'a'
    assert search(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == 'b'
    return 'tests pass'

print(test())

tests pass
